In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
cols = ['a1', 'a2', 'a3', 'a4', 'a5']

In [3]:
from CrossingBuilder import _build_crossing_prototypes

In [4]:
print(_build_crossing_prototypes(cols, 4))

[['a1', 'a1', 'a1', 'a1', 'a1'], ['a1', 'a1', 'a1', 'a1', 'a2'], ['a1', 'a1', 'a1', 'a1', 'a3'], ['a1', 'a1', 'a1', 'a1', 'a4'], ['a1', 'a1', 'a1', 'a1', 'a5'], ['a1', 'a1', 'a1', 'a2', 'a2'], ['a1', 'a1', 'a1', 'a2', 'a3'], ['a1', 'a1', 'a1', 'a2', 'a4'], ['a1', 'a1', 'a1', 'a2', 'a5'], ['a1', 'a1', 'a1', 'a3', 'a3'], ['a1', 'a1', 'a1', 'a3', 'a4'], ['a1', 'a1', 'a1', 'a3', 'a5'], ['a1', 'a1', 'a1', 'a4', 'a4'], ['a1', 'a1', 'a1', 'a4', 'a5'], ['a1', 'a1', 'a1', 'a5', 'a5'], ['a1', 'a1', 'a2', 'a2', 'a2'], ['a1', 'a1', 'a2', 'a2', 'a3'], ['a1', 'a1', 'a2', 'a2', 'a4'], ['a1', 'a1', 'a2', 'a2', 'a5'], ['a1', 'a1', 'a2', 'a3', 'a3'], ['a1', 'a1', 'a2', 'a3', 'a4'], ['a1', 'a1', 'a2', 'a3', 'a5'], ['a1', 'a1', 'a2', 'a4', 'a4'], ['a1', 'a1', 'a2', 'a4', 'a5'], ['a1', 'a1', 'a2', 'a5', 'a5'], ['a1', 'a1', 'a3', 'a3', 'a3'], ['a1', 'a1', 'a3', 'a3', 'a4'], ['a1', 'a1', 'a3', 'a3', 'a5'], ['a1', 'a1', 'a3', 'a4', 'a4'], ['a1', 'a1', 'a3', 'a4', 'a5'], ['a1', 'a1', 'a3', 'a5', 'a5'], ['a1', 